<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing(%2Bprc_lmt)_MondayOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.
#### 엑셀로 저장할 때 파라미터를 제외한 문구들(맨 위에 화면구성, 맨 아래에 주석 등)을 지우고 작업하는게 편합니다

In [111]:
# 파일 업로드
# 예제파일 2개(이론가파람_SK하이닉스, 상하한가) 파일을 https://github.com/SophieShin/MachineLearning/tree/main 에서 다운 받을 수 있습니다.

from google.colab import files

uploaded = files.upload()

Saving 코스피200옵션미니.csv to 코스피200옵션미니.csv


#### 아래 uploaded['이론가파람_SK하이닉스.csv'] 부분에서 한글 부분을 자신이 저장한 파일 이름으로 변경합니다

In [112]:
import io
df = pd.read_csv(io.BytesIO(uploaded['코스피200옵션미니.csv']), encoding='utf-8', thousands = ',')
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,C 202307 225.0,콜,2023-07-13,225.0,331.53,11,3.75,0.000000,0.970,106.94
1,C 202307 227.5,콜,2023-07-13,227.5,331.53,11,3.75,0.000000,0.970,104.48
2,C 202307 230.0,콜,2023-07-13,230.0,331.53,11,3.75,0.000000,0.970,102.01
3,C 202307 232.5,콜,2023-07-13,232.5,331.53,11,3.75,0.000000,0.970,99.57
4,C 202307 235.0,콜,2023-07-13,235.0,331.53,11,3.75,0.000000,0.970,97.14
...,...,...,...,...,...,...,...,...,...,...
707,P 202312 405.0,풋,2023-12-14,405.0,331.53,165,3.75,0.808213,0.144,67.82
708,P 202312 410.0,풋,2023-12-14,410.0,331.53,165,3.75,0.808213,0.144,72.64
709,P 202312 415.0,풋,2023-12-14,415.0,331.53,165,3.75,0.808213,0.144,77.48
710,P 202312 420.0,풋,2023-12-14,420.0,331.53,165,3.75,0.808213,0.144,82.34


In [113]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [114]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

In [115]:
# 풋옵션 이론가 산출 파라미터만 남기기
df2 = df[df['선물옵션구분'] == '풋']

In [116]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션구분'] == '콜']

#### 1. 콜옵션 이론가 구하기

In [117]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,C 202307 225.0,콜,2023-07-13,225.0,331.53,11,0.0375,0.000000,0.97,106.94
1,C 202307 227.5,콜,2023-07-13,227.5,331.53,11,0.0375,0.000000,0.97,104.48
2,C 202307 230.0,콜,2023-07-13,230.0,331.53,11,0.0375,0.000000,0.97,102.01
3,C 202307 232.5,콜,2023-07-13,232.5,331.53,11,0.0375,0.000000,0.97,99.57
4,C 202307 235.0,콜,2023-07-13,235.0,331.53,11,0.0375,0.000000,0.97,97.14
...,...,...,...,...,...,...,...,...,...,...
351,C 202312 405.0,콜,2023-12-14,405.0,331.53,165,0.0375,0.808213,0.15,0.46
352,C 202312 410.0,콜,2023-12-14,410.0,331.53,165,0.0375,0.808213,0.15,0.32
353,C 202312 415.0,콜,2023-12-14,415.0,331.53,165,0.0375,0.808213,0.15,0.24
354,C 202312 420.0,콜,2023-12-14,420.0,331.53,165,0.0375,0.808213,0.15,0.17


In [118]:
# 이론가격을 구하기 위해 data 자료타입 변경

S = list(map(int,df['기초자산\r\r\n기준가격'].tolist()))
K = list(map(int,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(int,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [119]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\r\r\n기준가격']*1.2
lower_lmt =  df['기초자산\r\r\n기준가격']*0.8
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [120]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [121]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [122]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  call(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)


In [123]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = call(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [124]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(call_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']
df['상한가'] = upper_limit
df['하한가'] = lower_limit

In [125]:
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,C 202307 225.0,콜,2023-07-13,225.0,331.53,11,0.0375,0.000000,0.97,106.94,106.417216,0.522784,174.117038,40.254137
1,C 202307 227.5,콜,2023-07-13,227.5,331.53,11,0.0375,0.000000,0.97,104.48,104.445488,0.034512,172.239099,38.256396
2,C 202307 230.0,콜,2023-07-13,230.0,331.53,11,0.0375,0.000000,0.97,102.01,101.487896,0.522104,169.422191,35.259785
3,C 202307 232.5,콜,2023-07-13,232.5,331.53,11,0.0375,0.000000,0.97,99.57,99.533314,0.036686,167.544252,33.262044
4,C 202307 235.0,콜,2023-07-13,235.0,331.53,11,0.0375,0.000000,0.97,97.14,96.616444,0.523556,164.776284,30.265432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,C 202312 405.0,콜,2023-12-14,405.0,331.53,165,0.0375,0.808213,0.15,0.46,0.472519,-0.012519,31.350968,0.000000
352,C 202312 410.0,콜,2023-12-14,410.0,331.53,165,0.0375,0.808213,0.15,0.32,0.329079,-0.009079,29.079598,0.000000
353,C 202312 415.0,콜,2023-12-14,415.0,331.53,165,0.0375,0.808213,0.15,0.24,0.245825,-0.005825,27.205272,0.000000
354,C 202312 420.0,콜,2023-12-14,420.0,331.53,165,0.0375,0.808213,0.15,0.17,0.174917,-0.004917,25.330947,0.000000


In [126]:
df.to_excel('미니옵션콜_검증용.xlsx')

In [127]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


#### 2. 풋옵션 이론가 구하기

In [128]:
# null값 제거
df2 = df2.dropna(axis = 1)
df2

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
356,P 202307 225.0,풋,2023-07-13,225.0,331.53,11,0.0375,0.000000,0.030,0.00
357,P 202307 227.5,풋,2023-07-13,227.5,331.53,11,0.0375,0.000000,0.030,0.00
358,P 202307 230.0,풋,2023-07-13,230.0,331.53,11,0.0375,0.000000,0.030,0.00
359,P 202307 232.5,풋,2023-07-13,232.5,331.53,11,0.0375,0.000000,0.030,0.00
360,P 202307 235.0,풋,2023-07-13,235.0,331.53,11,0.0375,0.000000,0.030,0.00
...,...,...,...,...,...,...,...,...,...,...
707,P 202312 405.0,풋,2023-12-14,405.0,331.53,165,0.0375,0.808213,0.144,67.82
708,P 202312 410.0,풋,2023-12-14,410.0,331.53,165,0.0375,0.808213,0.144,72.64
709,P 202312 415.0,풋,2023-12-14,415.0,331.53,165,0.0375,0.808213,0.144,77.48
710,P 202312 420.0,풋,2023-12-14,420.0,331.53,165,0.0375,0.808213,0.144,82.34


In [129]:
#인덱스 초기화
df2.reset_index(inplace = True)

In [130]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(int,df2['기초자산\r\r\n기준가격'].tolist()))
K = list(map(int,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(int,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [131]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\r\r\n기준가격']*0.8
lower_lmt =  df['기초자산\r\r\n기준가격']*1.8
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [132]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [133]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [134]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  put(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)

In [135]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = put(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [136]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(put_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df2['theo_price_검증용'] = call_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']
df2['상한가'] = upper_limit
df2['하한가'] = lower_limit

In [137]:
df2

,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,356,P 202307 225.0,풋,2023-07-13,225.0,331.53,11,0.0375,0.000000,0.030,0.00,0.000000,0.000000,0.000000,0.0
1,357,P 202307 227.5,풋,2023-07-13,227.5,331.53,11,0.0375,0.000000,0.030,0.00,0.000000,0.000000,0.000000,0.0
2,358,P 202307 230.0,풋,2023-07-13,230.0,331.53,11,0.0375,0.000000,0.030,0.00,0.000000,0.000000,0.000000,0.0
3,359,P 202307 232.5,풋,2023-07-13,232.5,331.53,11,0.0375,0.000000,0.030,0.00,0.000000,0.000000,0.000000,0.0
4,360,P 202307 235.0,풋,2023-07-13,235.0,331.53,11,0.0375,0.000000,0.030,0.00,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,707,P 202312 405.0,풋,2023-12-14,405.0,331.53,165,0.0375,0.808213,0.144,67.82,67.556780,0.263220,133.569437,0.0
352,708,P 202312 410.0,풋,2023-12-14,410.0,331.53,165,0.0375,0.808213,0.144,72.64,72.365996,0.274004,138.434314,0.0
353,709,P 202312 415.0,풋,2023-12-14,415.0,331.53,165,0.0375,0.808213,0.144,77.48,77.209410,0.270590,143.299191,0.0
354,710,P 202312 420.0,풋,2023-12-14,420.0,331.53,165,0.0375,0.808213,0.144,82.34,82.062771,0.277229,148.164068,0.0


In [138]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


In [140]:
df2.to_excel('미니옵션풋_검증용.xlsx')